<a href="https://colab.research.google.com/github/yaroslavtsepkov/APC/blob/main/monte_carlo/pi_monte_carlo_numpy_and_pycuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda

In [ ]:
import numpy as np
import time
import pandas as pd

import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from  pycuda import  driver
from pycuda.compiler import SourceModule
from pycuda.curandom import rand as curand

In [ ]:
def genData(p):
    x_gpu = curand((p,), dtype=np.double) 
    y_gpu = curand((p,), dtype=np.double)
    x = x_gpu.get().astype(np.double)
    y = y_gpu.get().astype(np.double)
    return x, y, p

In [ ]:
x, y, p = genData(p=1024 * 1024)

In [ ]:
def cpu_pi_monte_carlo(x, y, p):
    cpu_count = 0
    gen_cpu = [cpu_count + 1 for i in range(p) if (x[i] ** 2 + y[i] ** 2) < 1]
    result = 4/p * sum(gen_cpu)
    return result

In [ ]:
mod = SourceModule("""
                __global__ void gpu_pi_monte_carlo(double *x, double *y, double *count) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x; 
        int threadCount = gridDim.x * blockDim.x;
        int p = 1024 * 1024;
        int count_gpu = 0;
        for (int i = idx; i < p; i += threadCount) {
                if (x[i] * x[i] + y[i] * y[i] < 1) {
                        count_gpu++;
                }
        }
        atomicAdd(count , count_gpu);
}
""")

In [ ]:
t = time.time()
cpu_result = cpu_pi_monte_carlo(x, y, p)
cpu_time = time.time() - t

In [ ]:
gpu_result = gpuarray.zeros((1,), dtype=np.double)
gpu_result  = gpu_result.get()

In [ ]:
gpu_pi_monte_carlo = mod.get_function("gpu_pi_monte_carlo")
t = time.time()
gpu_pi_monte_carlo(driver.In(x), driver.In(y),  driver.Out(gpu_result), np.int32(p), block = (128, 1, 1), grid =(int(p/(128 * 128)), 1))
driver.Context.synchronize()
gpu_time = time.time() - t
gpu_result =  gpu_result[0] * 4/p

In [ ]:
df = pd.DataFrame(data={
    'CPU':{
        'PI':cpu_result,
        'Time, ms': round(cpu_time, 5),
    },
    'pyCUDA':{
        'PI':result_gpu,
        'Time, ms': round(gpu_time, 5),
    }
})

In [ ]:
df

In [ ]:
df.to_csv()